# Instala las librerias requeridas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# Sección nueva

In [ ]:
!pip install gmplot geopy
!pip install basemap
import pandas as pd
import numpy as np
import copy
import gmplot   
from random import randint, random, shuffle

# Crea clases para almacenar la solución

Crea las clases necesarias para almacenar los resultados e imprimirlos en el mapa

* **Clase Node**: Representa la información de cada nodo (i.e., id, latitud, longitud y demanda)
* **Clase Cluster**: Representa la información de cada cluster. Un cluster se entiende como un conjunto de nodos que forman una zona. Cada cluster tiene asociado:
  * *Centro*: se define como el nodo que se asumirá como el centro del cluster
  * *Lista de nodos*: Lista que agrupa todos los nodos pertenecientes al cluster
  * Medidas de desempeño del cluster:
    * *distAllToAll*: Suma de la distancia entre cada par de nodos del cluster
    * *distAllToCent*: Suma de la disntacia de todos los nodos del cluster respecto al nodo centro del cluster
    * *load*: Suma de la demanda de todos los nodos del cluster

* **Clase Solution**: Representa una solución completa y se compone:
  * *nClusters*: número de clusters
  * *clusters_list*: Listado de los clusters
  * *objectiveValue*: almacena el valor de la función objetivo. De momento se consideran tres objetivos:
    * sumAllToAll: Suma de todas las distancias  entre cada par de nodos pertencientes a un mismo cluster
    * sumAllToCenter: Suma de la distancia de cada uno los nodos al centro de su respectivo centro
    * loadRange: Diferencia entre el cluster con mayor demanda acumulada y el de menor demanda acumulada


In [ ]:
class Node:
  def __init__(self, id, lat=0, long=0, demand = 0):
    self.id = id
    self.lat = lat
    self.long = long
    self.demand = demand
 
 
class Cluster:
  def __init__(self, center):
    self.center = center
    self.node_list = []
    self.node_list.append(center)
    self.distAllToAll = []
    self.distAllToCent = []
    self.load = 0
 
  def get_measures(self, distances):
    # Distances all to center
    for node_id in self.node_list:
      self.distAllToCent.append(distances[(self.center.id, node_id.id)])
 
    # Distances all pair of nodes
    # It assumes non symmetric distances but also work for symmetric
    for node1 in self.node_list:
      for node2 in self.node_list:
        self.distAllToAll.append(distances[(node1.id, node2.id)])
 
    # get cluster load
    self.load = np.sum([node.demand for node in self.node_list])
 
 
class Solution:
  def __init__(self, nClusters):
    self.nClusters = nClusters
    self.clusters_list =[]
    self.objectiveValue = 0
 
  def get_objvalue(self, obj_function):
    def sum_alltocenter():
      s = np.sum([cluster.distAllToCent for cluster in self.clusters_list])
      return s
 
    def sum_alltoall():
      s = np.sum([cluster.distAllToAll for cluster in self.clusters_list])
      return s
 
    def load_range():
      load_max = self.load = np.max([cluster.load for cluster in self.clusters_list])
      load_min = self.load = np.min([cluster.load for cluster in self.clusters_list])
      return load_max - load_min
 
    # Dictionary that compiles options for the objective function
    functions = {
      'sumAllToCenter': sum_alltocenter,
      'sumAllToAll': sum_alltoall,
      'loadRange': load_range
    }
 
    # computes the objective with the given objective function
    obj = functions[obj_function]()
    return obj

# Generación de la instancia

Actualmente las descarga directamente desde el drive del curso, pero usted podría cargar sus propias instancias, tenga en cuenta que el archivo debe tener el formato dado

## Lectura de datos

In [ ]:
!gdown --id "1CTWhEQcKG-nMzwPodsgM4EdGMvKoRADp" # small instance
!gdown --id "1gR2ZH0savY_9Y9teBIMQ5tYw3ydVYpIQ" # large instance
!gdown --id "1oy-FuS4gErAPPYeabu78guiou7Xy-umM" # Paleta de colores para imprimir la solución en googlemaps

Debe escogerse cuál instancia se resolverá

In [ ]:
df = pd.read_csv('data_small.csv')
#df = pd.read_csv('data_large.csv')
colors_df = pd.read_csv('colors.csv') # convierte la paleta de colores en lista
colors_list = colors_df['colors'].tolist()

## Formato de la instancia

Los datos quedan almacenados en un dataframe

In [ ]:
colors_df

In [ ]:
df.head(5)

Adicionalmente, almacena los datos en una lista de nodos. Usando la clase nodo se definio anteriormente

In [ ]:
nodes = []
for row in range(len(df.index)):
  node = Node(int(df.loc[row, "id"]),
                  df.loc[row, "latitude"],
                  df.loc[row, "longitude"],
                  df.loc[row, "demand"])
  nodes.append(node)

In [ ]:
nodes

Calcula las distancias. Note que esto se almacenará como un diccionario cuya clave es una tupla formada por el id del nodo origen y el nodo destino `(id_origen, id_destino)` y cuyo valor es la distancia geodesica. 

**Nota:** Para la instancia grande esto puede tomar un tiempo considerable

In [ ]:
# Computes a dictionary of distances
from geopy.distance import distance
distances = {}
# computes the distance for each pair of nodes
for node1 in nodes:
  for node2 in nodes:
    d = distance((node1.lat, node1.long), (node2.lat, node2.long)).m
    key = (node1.id, node2.id)
    distances[key] = d
    #break
  #break

In [ ]:
distances

# Obtener solución

## 1. Solución óptima

Usted debería implementar a continuación el modelo de optimización que formuló para el problema, haciendo para ello uso de los datos de entrada 
* El dataframe `df` o la lista de `nodes` 
* El diccionario de distancias  `distances` 




In [ ]:
!pip install pyomo 
import pyomo.environ as pyo
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition

In [ ]:
df.set_index("id", inplace=True)

In [ ]:
df

# Modelo concreto

In [ ]:
# Sets
model = pyo.ConcreteModel()
model.C = pyo.Set(initialize=df.index, ordered=True, doc="Conjunto de colegios")

# Parámetros
model.dstn = pyo.Param(model.C, model.C, domain=pyo.NonNegativeReals, initialize=distances)
model.dmn = pyo.Param(model.C, domain=pyo.NonNegativeIntegers, initialize=df["demand"].to_dict(), doc="demanda por colegio")
model.k = pyo.Param(initialize=5, domain=pyo.NonNegativeIntegers, doc="número de clusters")
model.gamma = pyo.Param(initialize=0.1, domain=pyo.NonNegativeReals, doc="Desviación permitida")
# model.t_medio = pyo.Param(domain=pyo.NonNegativeReals, initialize=sum(model.dmn)/model.k.value, doc="Tamaño medio del cluster")

In [ ]:
# model.t_medio.display()

In [ ]:
# Variables de decisión
model.x = pyo.Var(model.C, model.C, within=pyo.Binary, doc="Asignación de colegios a acopios")
model.y = pyo.Var(model.C, within=pyo.Binary, doc="Designación de colegios como acopios")

In [ ]:
# Función objetivo
def obj_function(model):
  return sum(sum(model.dstn[i,j]*model.x[i,j] for i in model.C) for j in model.C)

model.objective = pyo.Objective(rule=obj_function, sense=pyo.minimize,
                                doc="cost function")

In [ ]:
# Restricciones

def restriccion2(model):
  return sum(model.y[i] for i in model.C) == model.k

model.r2 = pyo.Constraint(rule=restriccion2)

def restriccion3(model, i):
  return sum(model.dmn[j]*model.x[j,i] for j in model.C) <= (1+model.gamma)*model.y[i]*((sum(model.dmn[i] for i in model.C))/model.k)

model.r3 = pyo.Constraint(model.C, rule=restriccion3)


def restriccion4(model, i):
  return sum(model.dmn[j]*model.x[j,i] for j in model.C) >= (1-model.gamma)*model.y[i]*((sum(model.dmn[i] for i in model.C))/model.k)

model.r4 = pyo.Constraint(model.C, rule=restriccion4)

def restriccion1(model, i):
    return sum(model.x[i,j] for j in model.C) == 1

model.r1 = pyo.Constraint(model.C, rule=restriccion1)

In [ ]:
opt = pyo.SolverManagerFactory('neos')
results = opt.solve(model, opt='cplex', tee=True)


In [ ]:
model.y.display()

In [ ]:
model.x.display()

## 2. Solución ejemplo

A modo de ejemplo, se construirá una solución aleatoria. Asumiendo se obtuvo una solución óptima con el software, se dispone de la siguiente información:
* Número de clusters
* Nodo que será el centro de cada cluster 
* Lista de nodos de cada cluster

En nuestro caso asignaremos los primeros $k$ nodos como centros de los clusters y despues aleatoriamente asignaremos un nodo a cada cluster hasta asignarlos todos. Para el caso en el que se dispone la solución óptima la solución se crearía con dicha información

In [ ]:
# number of clusters
k = 4
# Creates solution
solution = Solution(k)
 
# Creates a deep copy of list of nodes
nodes_copy = copy.deepcopy(nodes)
# order the copied list randomly
shuffle(nodes_copy)
 
 
# Create clusters assigned the first n_cluster as centers
for i in range(solution.nClusters):
  node = nodes_copy.pop(0)
  cluster = Cluster(node)
  solution.clusters_list.append(cluster)
 
# Assigns the remaining nodes to the clusters
count = 0
while len(nodes_copy) > 0:
  node = nodes_copy.pop(0)
  id_cluster = count % solution.nClusters
  solution.clusters_list[id_cluster].node_list.append(node)
  count += 1
 
# Computes cluster measurements
for cluster in solution.clusters_list:
  cluster.get_measures(distances)

# Visualizar solución

Una vez construida la solución con la información proveida por el optimizador se podrá:

## Listar los cluster

In [ ]:
# print cluster
for cluster in solution.clusters_list:
  cluster_nodes = [node.id for node in cluster.node_list]
  print("Centro: ", cluster.center.id,
        " Nodos: ", cluster_nodes)

## Imprimir las medidas de desempeño

In [ ]:
# Get objective functions
print("Suma total de la distancia al centro: ", solution.get_objvalue("sumAllToCenter"))
print("Suma total de la distancia entre nodos de un mismo cluster: ", solution.get_objvalue("sumAllToAll"))
print("Diferencia entre demandas acumuladas máxima y minima: ", solution.get_objvalue("loadRange"))

## Generar mapa en html

In [ ]:
# get the coordinates of the first cluster to center the map on it
focus_lat = solution.clusters_list[0].center.lat
focus_long = solution.clusters_list[0].center.long
 
# Creates a map object
gmap3 = gmplot.GoogleMapPlotter(focus_lat, focus_long, 12.9)
# Set key google API
# Read documentation of google API to get your own key
# Otherwise it will with a temporary key
gmap3.apikey = " "
 
# Prints clusters
count_colors = 0
for cluster in solution.clusters_list:
  color_id = count_colors % len(colors_list)
  latitude_list = []
  longitude_list = []
  for node in cluster.node_list:
    latitude_list.append(node.lat)
    longitude_list.append(node.long)
  gmap3.scatter(latitude_list, longitude_list, colors_list[color_id], size=90, marker=False)
  count_colors += 1
 
gmap3.draw('map.html')

Descarga el mapa para ser abierto con algun navegador 

In [ ]:
from google.colab import files
files.download('map.html')